In [1]:
print("ok")

ok


In [2]:
%pwd

's:\\project\\ayurveda\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

's:\\project\\ayurveda'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Extract the data from the pdf File
#Tabnine: Edit | Test | Explain | Document | Ask
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents



In [9]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
#extracted_data

[Document(metadata={'source': 'Data\\Encyclopedia Of Herbal Medicine_2.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'Data\\Encyclopedia Of Herbal Medicine_2.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': 'Data\\Encyclopedia Of Herbal Medicine_2.pdf', 'page': 2}, page_content='HERBAL\nMEDICINE\nENCYCLOPEDIA OF\nUS_001_003_Half_Full_title.indd   1 26/02/16   4:42 pm\n'),
 Document(metadata={'source': 'Data\\Encyclopedia Of Herbal Medicine_2.pdf', 'page': 3}, page_content='US_001_003_Half_Full_title.indd   2 26/02/16   4:42 pm'),
 Document(metadata={'source': 'Data\\Encyclopedia Of Herbal Medicine_2.pdf', 'page': 4}, page_content='HERBAL\nMEDICINE\nENCYCLOPEDIA OF\nAndrew chevallier\nfnimh\nUS_001_003_Half_Full_title.indd   3 26/02/16   4:42 pm\n'),
 Document(metadata={'source': 'Data\\Encyclopedia Of Herbal Medicine_2.pdf', 'page': 5}, page_content=' \nDK UK \nEditor Toby Mann Senior Art Editor Elaine Hewson\nDesign Assistant Rehan Abdul\nUS Consul

In [14]:
#Split the Data into Text Chunks
#Tabnine: Edit | Test | Explain | Document | Ask
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks=text_split(extracted_data)
print("Lenght of Text Chunks", len(text_chunks))

Lenght of Text Chunks 5234


In [17]:
#text_chunks

In [42]:
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [37]:
#Download the Embeddings from Hugging Face
#Tabnine: Edit | Test | Explain | Document | Ask 
def download_hugging_face_embeddings():
    embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings 

In [44]:
embeddings= download_hugging_face_embeddings()

In [45]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [47]:
#query_result

In [151]:
from dotenv import load_dotenv
load_dotenv()

True

In [160]:
import os
import openai

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [161]:
Pinecone_Api_Key=os.environ.get('Pinecone_Api_key')
GROQ_API_KEY =os.environ.get('GROQ_API_KEY')

In [63]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=Pinecone_Api_Key)

index_name = "herbbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': 'c4107af4aab704468272978bafba6fc2;o=1', 'date': 'Fri, 21 Mar 2025 20:58:33 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
import os




os.environ["Pinecone_API_KEY"] = Pinecone_Api_Key
os.environ["GROQ_API_KEY"] = GROQ_API_KEY




In [65]:
#Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

KeyboardInterrupt: 

In [ ]:
#Load Existing index 

from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings

)

In [77]:
docsearch

In [78]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [79]:
retrieved_docs = retriver.invoke("What is Acne?")

In [59]:
retrieved_docs

[Document(metadata={'page': 118.0, 'source': 'Data\\The Complete Book of Ayurvedic Home Remedies.pdf'}, page_content='acne.\nVISUALIZATION\n. The root cause of acne is emotional stress. One eʃective way to relieve\nthat stress is visualization. Close your eyes and visualize that the acne is clearing up and\ngoing away—as if you are communicating with the tissues in your skin that are bursting\nout in acne. This works.\nA ɹnal suggestion: Avoid frequent looking in the mirror and feeling bad about the\nacne.\n \nAddictions\n \nSee also “\nSmoking\n”'),
 Document(metadata={'page': 117.0, 'source': 'Data\\The Complete Book of Ayurvedic Home Remedies.pdf'}, page_content='days—but this formula is quite effective in alleviating acne.\nDRINK ALOE VERA JUICE\n. You might try drinking half a cup of pure aloe vera juice twice a\nday.\nKEEP YOUR COLON CLEAN\n. Keeping your colon clean is important, in order to remove toxins\nfrom the body. You can easily accomplish this by taking the herb \namalak

In [ ]:
from langchain_openai import  OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [153]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant  for question-answering task. "
    "Use the following pieces of retrived context to answer "
    "the question. Inspire them to use homemade "
    "ayurvedic remedies.If you don't know the answer, say that "
    "you don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "[input]")
    ]
)

In [156]:
question_answer_chain = create_stuff_documents_chain(llm, prompt,)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [157]:
response = rag_chain.invoke({"input":"what is  Acne?"})
print(response["answer"])

To alleviate acne, try drinking half a cup of pure aloe vera juice twice a day and keep your colon clean by taking ½ to 1 teaspoon of amalaki powder daily. You can also apply melon on the affected area for relief. Additionally, practice visualization by closing your eyes and imagining the acne clearing up to reduce emotional stress.


In [159]:
response = rag_chain.invoke({"input":"what is  stats?"})
print(response["answer"])

To maintain health, consider homemade Ayurvedic remedies that balance your unique proportion of vata, pitta, and kapha. Factors like local diseases, diet, and habits influence your health, and Ayurvedic remedies can help. By using natural and high-quality herbal products, you can effectively maintain your balance and well-being.
